<a href="https://colab.research.google.com/github/MLLSCKR/ComputerVisionHomework/blob/main/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = "ComputerVisionLecture/Homework3"
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

import os
os.chdir('/content/drive/My Drive/{}'.format(FOLDERNAME))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import time
import cv2
import glob
import os
import numpy as np
import random
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.kernel_approximation import AdditiveChi2Sampler
from sklearn.svm import LinearSVC
from tqdm import tqdm

In [4]:
start_time = time.time()

'''
Parameters
'''
patch_stride = 16
K = 20

In [7]:
'''
Load Dataset
'''
def scene15():

    train_folders = glob.glob("SCENE-15/train/*")
    train_folders.sort()
    classes = dict()
    x_train = list()
    y_train = list()
    
    for index, folder in enumerate(train_folders):
        label = os.path.basename(folder)
        classes[label] = index
        paths = glob.glob(os.path.join(folder, "*"))
        for path in tqdm(paths):
            x_train.append(cv2.imread(path, 0))
            y_train.append(index)

    x_test = list()
    y_test = list()
    test_folders = glob.glob("SCENE-15/test/*")
    test_folders.sort()
    for folder in test_folders:
        label = os.path.basename(folder)
        index = classes[label]
        paths = glob.glob(os.path.join(folder, "*"))
        for path in tqdm(paths):
            x_test.append(cv2.imread(path, 0))
            y_test.append(index)

    return x_train, y_train, x_test, y_test, sorted(classes.keys())

print("Load Dataset ...")
x_train, y_train, x_test, y_test, labels_names = scene15()

random_indices = list(range(len(y_train)))
random.shuffle(random_indices)
x_train = np.array(x_train)[random_indices].tolist()
y_train = np.array(y_train)[random_indices].tolist()

Load Dataset ...


100%|██████████| 100/100 [00:00<00:00, 537.93it/s]

<class 'list'>



<ipython-input-7-e914e1ef2df2>:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train = np.array(x_train)[random_indices].tolist()


### 문제1. Bag-of-features 구현 및 Classifier 학습

#### 1-1. [특징 추출] 코드 상에서 주어진 Keypoint를 통하여, 각각의 이미지마다 특징을 추출하세요.

SIFT를 이용하여 특징을 추출하였습니다.
OpenCV package에서 제공하는 SIFT descriptor의 default 설정을 이용하였으며, 아래와 같이 descriptor를 선언하였습니다.

descriptor = cv2.xfeatures2d.SIFT_create()

In [16]:
'''
Extract Patches
'''
train_key_points = list()
train_feature_shapes = list()

for image in x_train:
    h, w = image.shape
    image_key_points = list()
    for x in range(0, w, patch_stride):
        for y in range(0, h, patch_stride):
            image_key_points.append(cv2.KeyPoint(x, y, patch_stride))
    train_key_points.append(image_key_points)
    train_feature_shapes.append((len(range(0, w, patch_stride)), (len(range(0, h, patch_stride)))))

print("train key points: ")
print(train_key_points[:5])

print("train feature shapes: ")
print(train_feature_shapes[:5])

train key points: 
[[< cv2.KeyPoint 0x7f6c02330930>, < cv2.KeyPoint 0x7f6c023307e0>, < cv2.KeyPoint 0x7f6c02333f60>, < cv2.KeyPoint 0x7f6c02330630>, < cv2.KeyPoint 0x7f6c02330660>, < cv2.KeyPoint 0x7f6c02330600>, < cv2.KeyPoint 0x7f6c02333e10>, < cv2.KeyPoint 0x7f6c02333ab0>, < cv2.KeyPoint 0x7f6c02330060>, < cv2.KeyPoint 0x7f6c02332340>, < cv2.KeyPoint 0x7f6c02333f00>, < cv2.KeyPoint 0x7f6c02330c30>, < cv2.KeyPoint 0x7f6c02331a70>, < cv2.KeyPoint 0x7f6c02332f70>, < cv2.KeyPoint 0x7f6c02332850>, < cv2.KeyPoint 0x7f6c02331a10>, < cv2.KeyPoint 0x7f6c02331410>, < cv2.KeyPoint 0x7f6c02330ea0>, < cv2.KeyPoint 0x7f6c02330ed0>, < cv2.KeyPoint 0x7f6c023335a0>, < cv2.KeyPoint 0x7f6c02332df0>, < cv2.KeyPoint 0x7f6c02333d80>, < cv2.KeyPoint 0x7f6c02333c60>, < cv2.KeyPoint 0x7f6c02333420>, < cv2.KeyPoint 0x7f6c02330de0>, < cv2.KeyPoint 0x7f6c02330720>, < cv2.KeyPoint 0x7f6c02330c60>, < cv2.KeyPoint 0x7f6c023321c0>, < cv2.KeyPoint 0x7f6c02330f30>, < cv2.KeyPoint 0x7f6c02333480>, < cv2.KeyPoint 0x7f

In [17]:
test_key_points = list()
test_feature_shapes = list()

for image in x_test:
    # Write Your Code Here #####################################################
    h, w = image.shape
    image_key_points = list()
    for x in range(0, w, patch_stride):
        for y in range(0, h, patch_stride):
            image_key_points.append(cv2.KeyPoint(x, y, patch_stride))
    test_key_points.append(image_key_points)
    test_feature_shapes.append((len(range(0, w, patch_stride)), (len(range(0, h, patch_stride)))))
    ############################################################################

print("test key poionts: ")
print(test_key_points[:5])

print("test feature shapes: ")
print(test_feature_shapes[:5])

test key poionts: 
[[< cv2.KeyPoint 0x7f6c02333a80>, < cv2.KeyPoint 0x7f6c02333e70>, < cv2.KeyPoint 0x7f6c023329d0>, < cv2.KeyPoint 0x7f6c02332640>, < cv2.KeyPoint 0x7f6c02333150>, < cv2.KeyPoint 0x7f6c02332970>, < cv2.KeyPoint 0x7f6c02333810>, < cv2.KeyPoint 0x7f6c02333840>, < cv2.KeyPoint 0x7f6c02332910>, < cv2.KeyPoint 0x7f6c023303f0>, < cv2.KeyPoint 0x7f6c02332be0>, < cv2.KeyPoint 0x7f6c05f59d40>, < cv2.KeyPoint 0x7f6c05f59d70>, < cv2.KeyPoint 0x7f6c02331bf0>, < cv2.KeyPoint 0x7f6c05f59dd0>, < cv2.KeyPoint 0x7f6c05f59e00>, < cv2.KeyPoint 0x7f6c05f59e30>, < cv2.KeyPoint 0x7f6c05f59e60>, < cv2.KeyPoint 0x7f6c05f59e90>, < cv2.KeyPoint 0x7f6c05f59ec0>, < cv2.KeyPoint 0x7f6c05f59ef0>, < cv2.KeyPoint 0x7f6c05f59f20>, < cv2.KeyPoint 0x7f6c05f5a220>, < cv2.KeyPoint 0x7f6c05f5a250>, < cv2.KeyPoint 0x7f6c05f5a280>, < cv2.KeyPoint 0x7f6c05f5a2b0>, < cv2.KeyPoint 0x7f6c05f59da0>, < cv2.KeyPoint 0x7f6c05f5bba0>, < cv2.KeyPoint 0x7f6c05f5a130>, < cv2.KeyPoint 0x7f6c05f5a160>, < cv2.KeyPoint 0x7f

In [18]:
'''
Extract Features
'''
# Write Your Code Here ############################################################
descriptor = cv2.xfeatures2d.SIFT_create()
###################################################################################
train_features = list()
index = 0
for image, key_points in zip(x_train, train_key_points):
    _, features = descriptor.detectAndCompute(image, None)
    train_features.append(features)
    index += 1
    print("Extract Train Features ... {:4d}/{:4d}".format(index, len(x_train)))

test_features = list()
index = 0
for image, key_points in zip(x_test, test_key_points):
    # Write Your Code Here ########################################################
    _, features = descriptor.detectAndCompute(image, None)
    ###############################################################################
    test_features.append(features)
    index += 1
    print("Extract Test Features ... {:4d}/{:4d}".format(index, len(x_test)))

Extract Train Features ...    1/1538
Extract Train Features ...    2/1538
Extract Train Features ...    3/1538
Extract Train Features ...    4/1538
Extract Train Features ...    5/1538
Extract Train Features ...    6/1538
Extract Train Features ...    7/1538
Extract Train Features ...    8/1538
Extract Train Features ...    9/1538
Extract Train Features ...   10/1538
Extract Train Features ...   11/1538
Extract Train Features ...   12/1538
Extract Train Features ...   13/1538
Extract Train Features ...   14/1538
Extract Train Features ...   15/1538
Extract Train Features ...   16/1538
Extract Train Features ...   17/1538
Extract Train Features ...   18/1538
Extract Train Features ...   19/1538
Extract Train Features ...   20/1538
Extract Train Features ...   21/1538
Extract Train Features ...   22/1538
Extract Train Features ...   23/1538
Extract Train Features ...   24/1538
Extract Train Features ...   25/1538
Extract Train Features ...   26/1538
Extract Train Features ...   27/1538
E

In [19]:
'''
Normalizing
'''
flattened_train_features = np.concatenate(train_features, axis=0)
pca = PCA(n_components=flattened_train_features.shape[-1], whiten=True)
pca.fit(flattened_train_features)
train_normalized_features = list()
index = 0
for features in train_features:
    features = pca.transform(features)
    train_normalized_features.append(features)
    index += 1
    print("Normalize Train Features ... {:4d}/{:4d}".format(index, len(train_features)))
test_normalized_features = list()
index = 0
for features in test_features:
    features = pca.transform(features)
    test_normalized_features.append(features)
    index += 1
    print("Normalize Test Features ... {:4d}/{:4d}".format(index, len(test_features)))

Normalize Train Features ...    1/1538
Normalize Train Features ...    2/1538
Normalize Train Features ...    3/1538
Normalize Train Features ...    4/1538
Normalize Train Features ...    5/1538
Normalize Train Features ...    6/1538
Normalize Train Features ...    7/1538
Normalize Train Features ...    8/1538
Normalize Train Features ...    9/1538
Normalize Train Features ...   10/1538
Normalize Train Features ...   11/1538
Normalize Train Features ...   12/1538
Normalize Train Features ...   13/1538
Normalize Train Features ...   14/1538
Normalize Train Features ...   15/1538
Normalize Train Features ...   16/1538
Normalize Train Features ...   17/1538
Normalize Train Features ...   18/1538
Normalize Train Features ...   19/1538
Normalize Train Features ...   20/1538
Normalize Train Features ...   21/1538
Normalize Train Features ...   22/1538
Normalize Train Features ...   23/1538
Normalize Train Features ...   24/1538
Normalize Train Features ...   25/1538
Normalize Train Features 

#### 1-2. [Bag-of-features 구현] K-means을 통해 구해진 codebook을 통하여, 각각의 이미지 특징을 encoding(histogram화 혹은 양자화) 하세요.

앞서 구한 image의 feature들을 양자화 하기 위해서 아래와 같은 code를 사용하였습니다.

        representations = np.zeros(dtype = np.int64, shape = (self.K))
        
        min_index = np.argmin(distances, axis = 1)
        for i in range(len(min_index)):
            idx = min_index[i]
            representations[idx] += 1

distances array에 저장되어 있는 각 cluster와의 거리 정보를 이용하여 가장 가까운 cluster를 찾고(np.argmin), 그 개수를 counting하는 방식으로 image feature를 encoding할 수 있었습니다.

In [8]:
'''
Make Codebook
'''
###################################################################################
# 아래의 코드의 빈 곳(None 부분)을 채우세요.
# None 부분 외의 부분은 가급적 수정 하지 말고, 주어진 형식에 맞추어
# None 부분 만을 채워주세요. 임의적으로 전체적인 구조를 수정하셔도 좋지만,
# 파이썬 코딩에 익숙 하지 않으시면, 가급적 틀을 유지하시는 것을 권장합니다.
# 1) 함수 encode 부분 안의 None 부분을 채우세요.
#    distances는 K means 알고리즘을 통해 얻어진 centroids, 즉 codewords(visual words)와
#    각 이미지의 특징들 간의 거리 입니다.
#    distances 값을 이용하여, features(# of keypoints, feature_dim)를
#    인코딩(histogram 혹은 quantization이라고도 함) 하세요.
#    인코딩된 결과인 representations은 (K)로 표현되어야 합니다.
#    이 때, K는 codewords의 개수입니다.
###################################################################################
class Codebook:

    def __init__(self, K):

        self.K = K

        self.kmeans = KMeans(n_clusters=K, verbose=False)

    def make_code_words(self, features):

        self.kmeans.fit(features)

    def encode(self, features, shapes):

        distances = self.kmeans.transform(features)

        # Write Your Code Here ########################################################
        representations = np.zeros(dtype=np.int64, shape=(len(distances), self.K))
        '''
        힌트- 방법1
        distances를 통해 representation에 대한 연산 진행
        np.argmin, np.arange, np.sum 등 라이브러리 사용
        '''
        '''
        힌트- 방법2
        nearest_centroid_indices = self.kmeans.predict(features)
        위와 같이 nearest_centroid_indices 활용한 연산도 가능
        '''
        # 적절한 방법을 통해 구현하고 왜 그런 방식을 사용하였는지 보고서 등에 작성해야 합니다.
        representations = np.zeros(dtype = np.int64, shape = (self.K))
        
        min_index = np.argmin(distances, axis = 1)
        for i in range(len(min_index)):
            idx = min_index[i]
            representations[idx] += 1
        ###############################################################################

        if np.array(representations).shape != (self.K, ):
            # representations는 반드시 (K) 차원을 가져야 합니다.
            # 해당 조건문은 잘못 구현했을 경우를 판단하기 위해 작성되었으며, 추후 문제없이 구현되었다면 지우셔도 됩니다.
            print(np.array(representations).shape)
            print("Your code may be wrong")

        return representations

In [21]:
print("Make Codebook ...")
flattened_normalized_train_features = pca.transform(flattened_train_features)
codebook = Codebook(K)
codebook.make_code_words(flattened_normalized_train_features)

'''
Encode Features
'''
train_encoded_features = list()
index = 0
for features, shapes in zip(train_normalized_features, train_feature_shapes):
    encoded_features = codebook.encode(features, shapes)
    train_encoded_features.append(encoded_features)
    index += 1
    print("Encoding Train Features ... {:4d}/{:4d}".format(index, len(train_normalized_features)))
test_encoded_features = list()
index = 0
for features, shapes in zip(test_normalized_features, test_feature_shapes):
    encoded_features = codebook.encode(features, shapes)
    test_encoded_features.append(encoded_features)
    index += 1
    print("Encoding Text Features ... {:4d}/{:4d}".format(index, len(test_normalized_features)))

Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Encoding Train Features ...    1/1538
Encoding Train Features ...    2/1538
Encoding Train Features ...    3/1538
Encoding Train Features ...    4/1538
Encoding Train Features ...    5/1538
Encoding Train Features ...    6/1538
Encoding Train Features ...    7/1538
Encoding Train Features ...    8/1538
Encoding Train Features ...    9/1538
Encoding Train Features ...   10/1538
Encoding Train Features ...   11/1538
Encoding Train Features ...   12/1538
Encoding Train Features ...   13/1538
Encoding Train Features ...   14/1538
Encoding Train Features ...   15/1538
Encoding Train Features ...   16/1538
Encoding Train Features ...   17/1538
Encoding Train Features ...   18/1538
Encoding Train Features ...   19/1538
Encoding Train Features ...   20/1538
Encoding Train Features ...   21/1538
Encoding Train Features ...   22/1538
Encoding Train Features ...   23/1538
Encoding Train Features ...   24/1538
Encoding Train Features ...   25/1538
Encoding Train Features ...   26/1538
Encoding Tra

In [22]:
'''
Approximate Kernel
'''
chi2sampler = AdditiveChi2Sampler(sample_steps=2)
chi2sampler.fit(train_encoded_features, y_train)
train_encoded_features = chi2sampler.transform(train_encoded_features)
test_encoded_features = chi2sampler.transform(test_encoded_features)

#### 1-3. [SVM을 통한 image 분류] Bag-of-features algorithm을 통해 얻어진 encoding 된 vector를 통해, SVM을 학습하세요.

문제1 에서 SVM을 통해 Classifier 학습이 구동되는지 확인하기 위하여 선언한 SVM model은 아래와 같습니다.


    model = LinearSVC(C=1, max_iter=100, verbose=False)

Regularization parameter(C)는 1로, max_iteration은 100회 학습을 진행하였습니다.

학습 결과 Train score: 0.20091, Test score: 0.18887을 기록하였습니다.

In [23]:
'''
Classify Images with SVM
'''
###################################################################################
# 아래의 코드의 빈 곳을 채우세요.
# 1) 아래의 model 부분에 sklearn 패키지를 활용하여, Linear SVM(SVC) 모델을 정의하세요.
#    처음에는 SVM의 parameter를 기본으로 설정하여 구동하시길 권장합니다.
#    구동 성공 시, SVM의 C 값과 max_iter 파라미터 등을 조정하여 성능 향상을 해보시길 바랍니다.
###################################################################################
# Write Your Code Here ############################################################
model = LinearSVC(C=1, max_iter=100, verbose=False)
###################################################################################

print("Classify Images ...")
# Write Your Code Here ############################################################
'''
위에서 정의한 model을 train data를 가지고 학습시켜보세요.
문제 1-3의 조건을 잘 읽고 해당되는 X와 y를 정의하면 됩니다.
'''
model.fit(train_encoded_features, y_train)

###################################################################################
train_score = model.score(train_encoded_features, y_train)
test_score = model.score(test_encoded_features, y_test)

elapsed_time = time.time() - start_time

Classify Images ...


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [24]:
'''
Print Results
'''
print()
print("=" * 90)
print("Train  Score: {:.5f}".format(train_score))
print("Test   Score: {:.5f}".format(test_score))
print("Elapsed Time: {:.2f} secs".format(elapsed_time))
print("=" * 90)


Train  Score: 0.46424
Test   Score: 0.35719
Elapsed Time: 302.34 secs


### 문제2. Parameter 조정을 통한 성능 개선
**문제 1에서 구현한 코드에는 여러 parameter가 존재합니다. 이를 자유롭게 조정하여 선은 개선을 시도해보세요. (자동화된 parameter search도 가능하며, K와 patch_stride 수정과 같이 memory 부족으로 큰 data를 사용하지 못하는 경우에는 절대적인 성능 지표보다 상대적인 성능 향상을 고려해주시면 됩니다.)**



In [9]:
def extract_patches(x_train, x_test, patch_stride):
    '''
    Extract Patches
    '''
    print("\tStart Extract Patches")

    train_key_points = list()
    train_feature_shapes = list()

    for image in x_train:
        h, w = image.shape
        image_key_points = list()
        for x in range(0, w, patch_stride):
            for y in range(0, h, patch_stride):
                image_key_points.append(cv2.KeyPoint(x, y, patch_stride))
        train_key_points.append(image_key_points)
        train_feature_shapes.append((len(range(0, w, patch_stride)), (len(range(0, h, patch_stride)))))

    test_key_points = list()
    test_feature_shapes = list()

    for image in x_test:
        # Write Your Code Here #####################################################
        h, w = image.shape
        image_key_points = list()
        for x in range(0, w, patch_stride):
            for y in range(0, h, patch_stride):
                image_key_points.append(cv2.KeyPoint(x, y, patch_stride))
        test_key_points.append(image_key_points)
        test_feature_shapes.append((len(range(0, w, patch_stride)), (len(range(0, h, patch_stride)))))
        ############################################################################

    return train_key_points, train_feature_shapes, test_key_points, test_feature_shapes

def extract_features(descriptor, x_train, train_key_points, x_test, test_key_points):
    '''
    Extract Features
    '''
    print("\tStart Extract Features")

    train_features = list()
    index = 0
    for image, key_points in zip(x_train, train_key_points):
        _, features = descriptor.detectAndCompute(image, None)
        train_features.append(features)
        index += 1

    test_features = list()
    index = 0
    for image, key_points in zip(x_test, test_key_points):
        # Write Your Code Here ########################################################
        _, features = descriptor.detectAndCompute(image, None)
        ###############################################################################
        test_features.append(features)
        index += 1

    '''
    Normalizing
    '''
    print("\tStart Normalizing")
    flattened_train_features = np.concatenate(train_features, axis=0)
    pca = PCA(n_components=flattened_train_features.shape[-1], whiten=True)
    pca.fit(flattened_train_features)
    train_normalized_features = list()
    index = 0
    for features in train_features:
        features = pca.transform(features)
        train_normalized_features.append(features)
        index += 1

    test_normalized_features = list()
    index = 0
    for features in test_features:
        features = pca.transform(features)
        test_normalized_features.append(features)
        index += 1

    return train_normalized_features, test_normalized_features, flattened_train_features, pca

def execution_training(model, descriptor, patch_stride, K, x_train, x_test):
    start_time = time.time()

    ###extract patches
    train_key_points, train_feature_shapes, test_key_points, test_feature_shapes = extract_patches(x_train, x_test, patch_stride)

    ###extract features
    train_normalized_features, test_normalized_features, flattened_train_features, pca = extract_features(descriptor, x_train, train_key_points, x_test, test_key_points)

    print("\tMake Codebook ...")
    flattened_normalized_train_features = pca.transform(flattened_train_features)
    codebook = Codebook(K)
    codebook.make_code_words(flattened_normalized_train_features)

    '''
    Encode Features
    '''
    print("\tStart Encoding Features")
    train_encoded_features = list()
    index = 0
    for features, shapes in zip(train_normalized_features, train_feature_shapes):
        encoded_features = codebook.encode(features, shapes)
        train_encoded_features.append(encoded_features)
        index += 1

    test_encoded_features = list()
    index = 0
    for features, shapes in zip(test_normalized_features, test_feature_shapes):
        encoded_features = codebook.encode(features, shapes)
        test_encoded_features.append(encoded_features)
        index += 1
    
    '''
    Approximate Kernel
    '''
    chi2sampler = AdditiveChi2Sampler(sample_steps=2)
    chi2sampler.fit(train_encoded_features, y_train)
    train_encoded_features = chi2sampler.transform(train_encoded_features)
    test_encoded_features = chi2sampler.transform(test_encoded_features)

    print("\tClassify Images ...")

    model.fit(train_encoded_features, y_train)

    train_score = model.score(train_encoded_features, y_train)
    test_score = model.score(test_encoded_features, y_test)

    elapsed_time = time.time() - start_time

    '''
    Print Results
    '''
    print()
    print("=" * 90)
    print("\tTrain  Score: {:.5f}".format(train_score))
    print("\tTest   Score: {:.5f}".format(test_score))
    print("\tElapsed Time: {:.2f} secs".format(elapsed_time))
    print("=" * 90)

#### 2-1. [feature extraction] faeture extraction algorithm과, keypoint를 만들 때의 patch_stride를 바꾸어보며 성능을 개선해보고, 성능의 증감의 이유를 분석해보세요.

실험결과: 


*   patch_stride(8): Train Score = 0.47919, Test Score = 0.37575
*   patch_stride(12): Train Score = 0.32705, Test Score = 0.26839
*   patch_stride(16): Train Score = 0.28804, Test Score = 0.23923
*   patch_stride(20): Train Score = 0.32120, Test Score = 0.26574
*   patch_stride(24): Train Score = 0.39662, Test Score = 0.33400

patch_stride가 8인 경우 가장 좋은 성능을 보였다. Train Score = 0.47919, Test Score = 0.37575

Feature Detection을 위한 patch size는 검출하고자 하는 특징에 따라 다르게 결정된다. 우리가 실험에 사용하고자 선택한 image dataset의 feature들을 가장 잘 추출할 수 있었던 patch stride 값이 8였기 때문으로 유추한다.


In [26]:
patch_stride_list = [8, 12, 16, 20, 24]

for idx, patch_stride in enumerate(patch_stride_list):
    print("problem 2-1. {} trial, patch_stride: {}".format(idx, patch_stride))

    descriptor = cv2.xfeatures2d.SIFT_create()
    model = LinearSVC(C=1, max_iter=100, verbose=False)
    K = 20
    execution_training(model, descriptor, patch_stride, K, x_train, x_test)

problem 2-1. 0 trial, patch_stride: 8
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



	Train  Score: 0.47919
	Test   Score: 0.37575
	Elapsed Time: 299.53 secs
problem 2-1. 1 trial, patch_stride: 12
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


...

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



	Train  Score: 0.32120
	Test   Score: 0.26574
	Elapsed Time: 252.93 secs
problem 2-1. 4 trial, patch_stride: 24
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...

	Train  Score: 0.39662
	Test   Score: 0.33400
	Elapsed Time: 309.10 secs


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#### 2-2. [K-means] K-means의 visual work 개수 K를 변경해보며 성능을 개선해보고, 성능 증감의 이유를 분석해보세요.

실험 결과:


*   K: 12, Train Score = 0.17555, Test Score = 0.13850
*   K: 16, Train Score = 0.40572, Test Score = 0.33002
*   K: 20, Train Score = 0.33225, Test Score = 0.27435
*   K: 24, Train Score = 0.31014, Test Score = 0.24785
*   K: 28, Train Score = 0.49740, Test Score = 0.32936

실험결과 k=28에서 Train Score = 0.40572, Test Score=0.32936로 좋은 성능을 보였다. K가 증가할 수록 성능이 증가하는 양상을 보이는 데, 이는 K가 증가할수록 classifer가 학습하는 data의 차원이 증가하여 더 고차원적인 분류기준을 학습할 수 있기 때문이라 생각한다.


In [28]:
K_list = [12, 16, 20, 24, 28]

for idx, K in enumerate(K_list):
    print("problem 2-2. {} trial, K: {}".format(idx, K))
    descriptor = cv2.xfeatures2d.SIFT_create()
    model = LinearSVC(C=1, max_iter=100, verbose=False)
    patch_stride = 8
    execution_training(model, descriptor, patch_stride, K, x_train, x_test)

problem 2-2. 0 trial, K: 12
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



	Train  Score: 0.17555
	Test   Score: 0.13850
	Elapsed Time: 213.18 secs
problem 2-2. 1 trial, K: 16
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


...

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



	Train  Score: 0.31014
	Test   Score: 0.24785
	Elapsed Time: 318.98 secs
problem 2-2. 4 trial, K: 28
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...

	Train  Score: 0.49740
	Test   Score: 0.32936
	Elapsed Time: 405.89 secs


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#### 2-3. [SVM] SVM의 학습 파라미터인 C, max_iter 값 등을 수정해보며 성능을 개선해보고, 성능의 증감의 이유를 분석해보세요.

실험 결과:


*   (C: 0.5, max_iter:25): Train Score = 0.43238, Test Score = 0.30616
*   (C: 0.5, max_iter:50): Train Score = 0.32640, Test Score = 0.26176
*   (C: 0.5, max_iter:100): Train Score = 0.54421, Test Score = 0.36978
*   (C: 0.5, max_iter:200): Train Score = 0.47984, Test Score = 0.34129
*   (C: 0.5, max_iter:400): Train Score = 0.61183, Test Score = 0.38370

*   (C: 0.75, max_iter:25): Train Score = 0.47464, Test Score = 0.35520
*   (C: 0.75, max_iter:50): Train Score = 0.46554, Test Score = 0.34924
*   (C: 0.75, max_iter:100): Train Score = 0.27893, Test Score = 0.23658
*   (C: 0.75, max_iter:200): Train Score = 0.48179, Test Score = 0.31743
*   (C: 0.75, max_iter:400): Train Score = 0.59233, Test Score = 0.39099

*   (C: 1, max_iter:25): Train Score = 0.39012, Test Score = 0.30219
*   (C: 1, max_iter:50): Train Score = 0.29974, Test Score = 0.23791
*   (C: 1, max_iter:100): Train Score = 0.40832, Test Score = 0.29689
*   (C: 1, max_iter:200): Train Score = 0.38817, Test Score = 0.29225
*   (C: 1, max_iter:400): Train Score = 0.51886, Test Score = 0.37707

*   (C: 1.25, max_iter:25): Train Score = 0.30884, Test Score = 0.22929
*   (C: 1.25, max_iter:50): Train Score = 0.45189, Test Score = 0.33665
*   (C: 1.25, max_iter:100): Train Score = 0.46684, Test Score = 0.34261
*   (C: 1.25, max_iter:200): Train Score = 0.37841, Test Score = 0.29622
*   (C: 1.25, max_iter:400): Train Score = 0.52406, Test Score = 0.34261

*   (C: 1.5, max_iter:25): Train Score = 0.23472, Test Score = 0.19218
*   (C: 1.5, max_iter:50): Train Score = 0.48309, Test Score = 0.32604
*   (C: 1.5, max_iter:100): Train Score = 0.38296, Test Score = 0.29490
*   (C: 1.5, max_iter:200): Train Score = 0.32120, Test Score = 0.24785
*   (C: 1.5, max_iter:400): Train Score = 0.52731, Test Score = 0.35852




실험결과 C:0.75, max_iter:400일 경우, Test Score=0.39099으로 가장 좋은 성능릉 보였다.

SVM 학습 parameter C는 Regularization parameter로, Strength of the regularization은 C에 반비례하는 관계를 지닌다.

때문에 max_iter값이 25인 경우, C값이 커질수록 test Score가 오히려 감소하는 경향을 보여준다. 이는 Regularization이 충분치 않아 overfitting되어 Test Score와 Train Score 간의 차이가 발생한 것이다.

동일한 C값 내에서 max_iter 값이 증가할 수록 Train Score와 Test Score 모두 증가하는 경향을 보였다. 

실험 결과 가장 적절한 C, Max_iteration값은 (C:0.75, max_iter:400)인 case에서 Train Score=0.59233, Test Score = 0.39099를 기록하였다.


In [29]:
C_list = [0.5, 0.75, 1, 1.25, 1.5]
max_iter_list = [25, 50, 100, 200, 400]

idx = 0
for C in C_list:
    for max_iter in max_iter_list:
        print("problem 2-3. {} trial, C: {}, max_iter : {}".format(idx, C, max_iter))
        descriptor = cv2.xfeatures2d.SIFT_create()
        model = LinearSVC(C=C, max_iter=max_iter, verbose=False)
        patch_stride = 8
        K = 28
        execution_training(model, descriptor, patch_stride, K, x_train, x_test)
        idx += 1

problem 2-3. 0 trial, C: 0.5, max_iter : 25
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...

	Train  Score: 0.43238
	Test   Score: 0.30616
	Elapsed Time: 399.45 secs
problem 2-3. 1 trial, C: 0.5, max_iter : 50
	Start Extract Patches


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


	Start Extract Features
	Start Normalizing
	Make Codebook ...


...

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(



	Train  Score: 0.32120
	Test   Score: 0.24785
	Elapsed Time: 363.25 secs
problem 2-3. 24 trial, C: 1.5, max_iter : 400
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...

	Train  Score: 0.52731
	Test   Score: 0.35852
	Elapsed Time: 383.30 secs


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


#### 2-4. 실험 결과를 토대로 Best Model을 선정하고, Best Model의 Confusion matrix를 그리고 분석하세요.

Best Model의 Confusion Matrix는 아래와 같다.

    [[28  4  1  4  5  2  5  4  2  9 11 12 22  0  0]
    [ 2 48  4 11  0  0  0  0  6  2 18  1  7  0  1]
    [ 0  1 72  0  0  0  0  0  2  0  2 19  4  0  0]
    [ 2  8  0 43 10  2  2  1  2  2  9  1 13  1  4]
    [ 6  3  0  5  5  0  6  1  2  4  1 25 38  4  0]
    [ 4  1  0  2  5 18  0  0  0  9  0 26 29  3  3]
    [14  1  0  0  3  2 16  4  1 14  1 14 26  1  3]
    [13  0  0  2  1  0  6  8  0  8  0 30 26  4  2]
    [ 1 12  4  4  0  0  0  2 35  0 15  8 18  1  0]
    [19  1  0  1  2  5 12  6  1 16  0 20 16  0  1]
    [ 3 20  4  4  0  0  0  0  9  0 36  7 13  1  3]
    [ 2  0  1  0  0  1  2  0  1  0  0 72 18  3  0]
    [ 2  0  0  0  0  1  0  2  2  2  1 24 64  0  2]
    [ 0  0  0  0  1  0  0  2  0  0  1 34  7 54  1]
    [ 4  3  1  6  1  5  0  1  0  2  4 12 31  1 29]]

Confusion Matrix를 보면 Class 별로 정답률의 차이가 있음을 알 수 있다.

가장 높은 정답률을 기록한 class는 3번째 Class('Forest')이다. 반면에 5번째 class('Industrial')의 경우 낮은 정답률을 보였다.

Class 별로 정답률의 차이를 보인 이유는 분류하고자 하는 class 별로 복잡한 정도가 다르기 때문이다. 실제 Industrial Folder 내의 image들을 열어 보면, 공통된 특징을 발견하기 굉장히 힘들다. 반면 Forest Folder 내의 image들은 유사한 특징을 발견하기 비교적 쉬움을 알 수 있다.

In [31]:
from sklearn.metrics import confusion_matrix

def best_model_confusion_matrix(model, descriptor, patch_stride, K, x_train, x_test):
    start_time = time.time()

    ###extract patches
    train_key_points, train_feature_shapes, test_key_points, test_feature_shapes = extract_patches(x_train, x_test, patch_stride)

    ###extract features
    train_normalized_features, test_normalized_features, flattened_train_features, pca = extract_features(descriptor, x_train, train_key_points, x_test, test_key_points)

    ###training models
    flattened_normalized_train_features = pca.transform(flattened_train_features)
    codebook = Codebook(K)
    codebook.make_code_words(flattened_normalized_train_features)

    train_encoded_features = list()
    index = 0
    for features, shapes in zip(train_normalized_features, train_feature_shapes):
        encoded_features = codebook.encode(features, shapes)
        train_encoded_features.append(encoded_features)
        index += 1

    test_encoded_features = list()
    index = 0
    for features, shapes in zip(test_normalized_features, test_feature_shapes):
        encoded_features = codebook.encode(features, shapes)
        test_encoded_features.append(encoded_features)
        index += 1
    
    chi2sampler = AdditiveChi2Sampler(sample_steps=2)
    chi2sampler.fit(train_encoded_features, y_train)
    train_encoded_features = chi2sampler.transform(train_encoded_features)
    test_encoded_features = chi2sampler.transform(test_encoded_features)

    model.fit(train_encoded_features, y_train)

    ###make confusion matrix
    y_pred = model.predict(test_encoded_features)

    cf = confusion_matrix(y_true = y_test, y_pred = y_pred)

    elapsed_time = time.time() - start_time

    '''
    Print Results
    '''
    print()
    print("=" * 90)
    print("confusion matrix!!!")
    print(cf)
    print("=" * 90)

In [32]:
print("problem 2-4. Best Model: patch_stride = 8, K=28, C=0.75, Max_iter=400")

descriptor = cv2.xfeatures2d.SIFT_create()
model = LinearSVC(C=0.75, max_iter=400, verbose=False)
patch_stride = 8
K = 28
best_model_confusion_matrix(model, descriptor, patch_stride, K, x_train, x_test)

problem 2-4. Best Model: patch_stride = 8, K=28, C=0.75, Max_iter=400
	Start Extract Patches
	Start Extract Features
	Start Normalizing


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(



confusion matrix!!!
[[28  4  1  4  5  2  5  4  2  9 11 12 22  0  0]
 [ 2 48  4 11  0  0  0  0  6  2 18  1  7  0  1]
 [ 0  1 72  0  0  0  0  0  2  0  2 19  4  0  0]
 [ 2  8  0 43 10  2  2  1  2  2  9  1 13  1  4]
 [ 6  3  0  5  5  0  6  1  2  4  1 25 38  4  0]
 [ 4  1  0  2  5 18  0  0  0  9  0 26 29  3  3]
 [14  1  0  0  3  2 16  4  1 14  1 14 26  1  3]
 [13  0  0  2  1  0  6  8  0  8  0 30 26  4  2]
 [ 1 12  4  4  0  0  0  2 35  0 15  8 18  1  0]
 [19  1  0  1  2  5 12  6  1 16  0 20 16  0  1]
 [ 3 20  4  4  0  0  0  0  9  0 36  7 13  1  3]
 [ 2  0  1  0  0  1  2  0  1  0  0 72 18  3  0]
 [ 2  0  0  0  0  1  0  2  2  2  1 24 64  0  2]
 [ 0  0  0  0  1  0  0  2  0  0  1 34  7 54  1]
 [ 4  3  1  6  1  5  0  1  0  2  4 12 31  1 29]]


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### 문제3. 알고리즘 개선
#### 3-1. 템플릿 코드를 기반으로 구현한 현재 모델은 분류를 위해 SVM을 사용하고 있다. Skleran package에서 SVM외의 다른 classfier를 2가지 이상 사용하여 실험을 진행하고 결과를 분석하세요.

SVM 이외에 Decision Tree, MLP classifier를 이용하여 실험을 진행하였다.

1. Decision Tree 실험 결과
    
    Train Score: 1.0
    Test Score: 0.27700

2. MLP 실험 결과
    
  (hidden layer: (3, 10))
    - Train Score: 0.41287
    - Test Score: 0.36315

  (hidden layer: (5, 10))
    - Train Score: 0.46879
    - Test Score: 0.38370
  
  (hidden layer: (7, 10))
    - Train Score: 0.50650
    - Test Score: 0.39695

  (hidden layer: (9, 10))
    - Train Score: 0.50000
    - Test Score: 0.41286
  
  (hidden layer: (15, 0))
    - Train Score: 0.51040
    - Test Score: 0.40358

Decision Tree를 사용하여 classification을 수행하였을 때 model이 overfitting 되어 있음을 확인할 수 있다.(Train Score=1.0, Test Score=0.27700) 별도의 parameter 설정 없이 Default 값들을 사용하여 학습을 진행하였는데, 가지치기와 같은 overfitting을 제어하기 위한 추가적인 설정을 추가한다면 overfitting을 제어할 수 있을 것으로 판단된다.

MLP를 사용하여 classification을 수행하였을 때, hidden layer 9개까지 test score가 증가하는 것을 관찰 할 수 있다. 그 이후부터는 model이 overfitting 되어 test score가 감소함을 확인할 수 있었다.

In [10]:
from sklearn import tree

print("problem 3-1. model: Decision Tree")
descriptor = cv2.xfeatures2d.SIFT_create()
model = tree.DecisionTreeClassifier()
patch_stride = 8
K = 28
execution_training(model, descriptor, patch_stride, K, x_train, x_test)

problem 3-1. model: Decision Tree
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...

	Train  Score: 1.00000
	Test   Score: 0.27700
	Elapsed Time: 336.83 secs


In [12]:
from sklearn.neural_network import MLPClassifier

print("problem 3-1. model: MLP, hidden layer sizes = (3, 10)")
descriptor = cv2.xfeatures2d.SIFT_create()
model = MLPClassifier(hidden_layer_sizes=(3, 10), random_state=1, max_iter = 400)
patch_stride = 8
K = 28
execution_training(model, descriptor, patch_stride, K, x_train, x_test)

problem 3-1. model: MLP, hidden layer sizes = (3, 10)
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...

	Train  Score: 0.41287
	Test   Score: 0.36315
	Elapsed Time: 380.50 secs


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


In [13]:
from sklearn.neural_network import MLPClassifier

print("problem 3-1. model: MLP, hidden layer sizes = (5, 10)")
descriptor = cv2.xfeatures2d.SIFT_create()
model = MLPClassifier(hidden_layer_sizes=(5, 10), random_state=1, max_iter = 400)
patch_stride = 8
K = 28
execution_training(model, descriptor, patch_stride, K, x_train, x_test)

problem 3-1. model: MLP, hidden layer sizes = (5, 10)
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...

	Train  Score: 0.46879
	Test   Score: 0.38370
	Elapsed Time: 335.13 secs


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


In [14]:
from sklearn.neural_network import MLPClassifier

print("problem 3-1. model: MLP, hidden layer sizes = (7, 10)")
descriptor = cv2.xfeatures2d.SIFT_create()
model = MLPClassifier(hidden_layer_sizes=(7, 10), random_state=1, max_iter = 400)
patch_stride = 8
K = 28
execution_training(model, descriptor, patch_stride, K, x_train, x_test)

problem 3-1. model: MLP, hidden layer sizes = (7, 10)
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...

	Train  Score: 0.50650
	Test   Score: 0.39695
	Elapsed Time: 330.94 secs


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


In [15]:
from sklearn.neural_network import MLPClassifier

print("problem 3-1. model: MLP, hidden layer sizes = (9, 10)")
descriptor = cv2.xfeatures2d.SIFT_create()
model = MLPClassifier(hidden_layer_sizes=(7, 10), random_state=1, max_iter = 400)
patch_stride = 8
K = 28
execution_training(model, descriptor, patch_stride, K, x_train, x_test)

problem 3-1. model: MLP, hidden layer sizes = (9, 10)
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...

	Train  Score: 0.50000
	Test   Score: 0.41286
	Elapsed Time: 389.30 secs


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (400) reached and the optimization hasn't converged yet.
  warnings.warn(


In [16]:
from sklearn.neural_network import MLPClassifier

print("problem 3-1. model: MLP, hidden layer sizes = (15, 10)")
descriptor = cv2.xfeatures2d.SIFT_create()
model = MLPClassifier(hidden_layer_sizes=(7, 10), random_state=1, max_iter = 400)
patch_stride = 8
K = 28
execution_training(model, descriptor, patch_stride, K, x_train, x_test)

problem 3-1. model: MLP, hidden layer sizes = (15, 10)
	Start Extract Patches
	Start Extract Features
	Start Normalizing
	Make Codebook ...


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


	Start Encoding Features
	Classify Images ...

	Train  Score: 0.51040
	Test   Score: 0.40358
	Elapsed Time: 388.07 secs
